In [45]:
from fastai_old.text import *
import html
import spacy 

spacy.load('en')

In [2]:
num_verses = 3
verses = []
chorus = ''

In [3]:
PATH=Path('data/')
LM_PATH=Path('data/model_lm/')
LM_PATH.mkdir(exist_ok=True)

In [18]:
itosv = pickle.load(open(LM_PATH/'tmp'/'itos-large-verse.pkl', 'rb'))
stoiv = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itosv)})
itosc = pickle.load(open(LM_PATH/'tmp'/'itos-large-chorus.pkl', 'rb'))
stoic = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itosc)})
vsv=len(itosv)
vsc=len(itosc)
vsv, vsc

(17190, 3642)

In [19]:
trn_lm = np.empty([10,2])
val_lm = np.empty([10,2])

In [20]:
em_sz,nh,nl = 400,1150,3
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7
wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
mdv = LanguageModelData(PATH, 1, vsv, trn_dl, val_dl, bs=bs, bptt=bptt)
mdc = LanguageModelData(PATH, 1, vsc, trn_dl, val_dl, bs=bs, bptt=bptt)

DATA: 0 70
DATA: 0 70


In [21]:
learnerv= mdv.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learnerc= mdc.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

#learner.model.load_state_dict(wgts)

INIT <fastai_old.text.LanguageModelData object at 0x7f7393f677f0>
INIT <fastai_old.text.LanguageModelData object at 0x7f7393f67780>


In [9]:
def generate_text_verse(m, s, itos, stoi, l=20):
    m[0].bs=1  # Set batch size to 1
    m.eval()  # Turn off dropout
    m.reset()  # Reset hidden state
    m[0].bs=bs  # Put the batch size back to what it was

    ss = s.lower().split()
    si = [stoi[w] for w in ss]
    t = torch.autograd.Variable(torch.cuda.LongTensor(np.array([si])))
    
    res,*_ = m(t)
    
    output = s + ' '
    count = 0
    while True:
        #n = res[-1].topk(5)[1]  # top word
        n = torch.multinomial(res[-1].exp(), 10)  # drawing from probability distribution
        n = n[1] if n.data[0]==0 else n[0]
        if itos[int(n)] == '\n' and count > l:
            output += "\n"
            break
        if not any (x in itos[int(n)] for x in ['xbos', 'xfld']):
            output += itos[int(n)] + ' '
        res,*_ = m(n.unsqueeze(0).unsqueeze(0))  # sometimes need an extra .unsqueeze(0)
        count += 1
    return output

In [22]:
learnerv.load('lm_30epochs-large-verse-set')
learnerc.load('lm_30epochs-large-chorus')
m=learnerv.model
for i in range(num_verses):
    verses.append(generate_text_verse(m, "this is the way I", itosv, stoiv, l=150))

m=learnerc.model
chorus = generate_text_verse(m, "this is the way I", itosc, stoic, l=100)

In [43]:
def clean_output(x):
    return x.replace(" n't", "n't").replace(" ' til", "@'til").replace(" ' cause", "@'cause").replace(
        " '", "'").replace("@'til", " 'til").replace("@'cause", " 'cause").replace(" ,", ",")

In [44]:
print('[Intro]')
for i in range(num_verses):
    print('\n[Verse ' + str(i + 1) + ']')
    print(clean_output(verses[i]))
    print('\n[Chorus]')
    print(clean_output(chorus))

[Intro]

[Verse 1]
this is the way I might go ape - baskets 
 from puerto rico to that chaining question 
 nowadays these hoes is imaginary 
 it's so much money that these haters couldn't even cross me 
 we had to pay the rent to keep my arm dry 
 them bitches can no longer brag, now my niggas flyin 
 counting a gras 'til they just get to tez 
 sean don, kobe bryant fiends 
 crap tables and it's all black, they don't want a crap 
 next time they see me they wanna beg me for they freaking ting 
 but wouldn't it be a comeback if i didn't list you 
 if you did what it did, what would you do if you knew me ? 
 1 i'm peepin' out all these halfway hoes 
 but it don't matter, i'm just on the pager 


[Chorus]
this is the way I swear to god, hail mary 
 i'll be your savior, oh god, oh mama 
 1 all my niggas - what it feel like ? ( oh ! ) 
 have ya dead or gone ( ah ! ) 
 have my bitch, dope - oh, my nigga get ( yo ! ) 
 tell my girlfriend, my bitch ! ( oh ! ) 
 ( oh, ya smile ! ) 
 do nt go ag